## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [4]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [5]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [6]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [7]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [8]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [9]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

chain = prompt | model

In [10]:
chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='{topic} 에 대해 쉽게 설명해주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x117bd6b50>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x117be3890>, root_client=<openai.OpenAI object at 0x117b10d90>, root_async_client=<openai.AsyncOpenAI object at 0x117bd6d10>, model_name='gpt-4o-mini', temperature=0.1, model_kwargs={}, openai_api_key=SecretStr('**********'))

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [11]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "인공지능 모델의 학습 원리"}

In [12]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

AIMessage(content='인공지능 모델의 학습 원리를 쉽게 설명하자면, 다음과 같은 단계로 나눌 수 있습니다.\n\n1. **데이터 수집**: 인공지능 모델은 학습하기 위해 많은 데이터를 필요로 합니다. 이 데이터는 이미지, 텍스트, 소리 등 다양한 형태일 수 있습니다.\n\n2. **데이터 전처리**: 수집한 데이터는 모델이 이해할 수 있는 형태로 가공해야 합니다. 예를 들어, 이미지의 크기를 조정하거나, 텍스트를 숫자로 변환하는 과정이 필요합니다.\n\n3. **모델 선택**: 학습할 모델을 선택합니다. 이는 신경망, 결정 트리, 서포트 벡터 머신 등 여러 가지 방법이 있을 수 있습니다. 각 모델은 특정한 문제에 더 적합할 수 있습니다.\n\n4. **학습**: 모델은 주어진 데이터를 바탕으로 패턴을 학습합니다. 이 과정에서 모델은 입력 데이터와 정답(라벨) 간의 관계를 파악하게 됩니다. 예를 들어, 고양이와 개의 이미지를 구분하는 모델은 각각의 특징을 학습하게 됩니다.\n\n5. **손실 함수**: 모델의 예측이 실제 정답과 얼마나 차이가 있는지를 측정하는 손실 함수를 사용합니다. 이 값을 최소화하는 방향으로 모델의 파라미터(가중치)를 조정합니다.\n\n6. **최적화**: 경량화된 알고리즘(예: 경사 하강법)을 사용하여 모델의 파라미터를 업데이트합니다. 이 과정을 반복하면서 모델은 점점 더 정확한 예측을 하게 됩니다.\n\n7. **검증**: 학습이 끝난 후, 새로운 데이터(검증 데이터)를 사용하여 모델의 성능을 평가합니다. 이 단계에서 모델이 실제로 잘 작동하는지를 확인합니다.\n\n8. **배포 및 사용**: 모델이 충분히 학습하고 검증되면, 실제 환경에 배포하여 사용합니다. 이후에도 새로운 데이터로 모델을 업데이트하거나 재학습할 수 있습니다.\n\n이러한 과정을 통해 인공지능 모델은 데이터를 기반으로 학습하고, 주어진 문제를 해결하는 능력을 갖추게 됩니다.', additional_kwargs={'refusal': None}, respons

In [15]:
chain.invoke("양자컴퓨팅")

AIMessage(content='양자컴퓨팅은 전통적인 컴퓨터와는 다른 방식으로 정보를 처리하는 컴퓨터 기술입니다. 전통적인 컴퓨터는 비트(bit)를 사용하여 정보를 0 또는 1의 형태로 저장하고 처리합니다. 반면, 양자컴퓨터는 양자 비트(큐비트, qubit)를 사용합니다.\n\n큐비트는 다음과 같은 두 가지 중요한 특성을 가지고 있습니다:\n\n1. **중첩(Superposition)**: 큐비트는 0과 1의 상태를 동시에 가질 수 있습니다. 즉, 큐비트는 여러 상태를 동시에 표현할 수 있어, 여러 계산을 동시에 수행할 수 있는 가능성을 제공합니다.\n\n2. **얽힘(Entanglement)**: 두 개 이상의 큐비트가 얽히면, 하나의 큐비트 상태가 다른 큐비트의 상태에 영향을 미치는 현상이 발생합니다. 이를 통해 큐비트 간의 상호작용을 통해 더 복잡한 계산을 수행할 수 있습니다.\n\n양자컴퓨터는 이러한 특성 덕분에 특정 문제를 전통적인 컴퓨터보다 훨씬 빠르게 해결할 수 있는 잠재력을 가지고 있습니다. 예를 들어, 양자컴퓨터는 복잡한 암호 해독, 최적화 문제, 양자 화학 시뮬레이션 등에서 큰 장점을 가질 수 있습니다.\n\n하지만 양자컴퓨팅은 아직 초기 단계에 있으며, 실용적인 양자컴퓨터를 만드는 데는 많은 기술적 도전과제가 남아 있습니다. 그럼에도 불구하고, 양자컴퓨터는 미래의 컴퓨팅 기술로 많은 기대를 받고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 359, 'prompt_tokens': 18, 'total_tokens': 377, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details

In [16]:
prompt2 = PromptTemplate.from_template("{topic}에 대하여 {how} 설명해주세요")

chain2 = prompt2 | model


In [18]:
chain2.invoke({"topic": "양자컴퓨팅", "how": "간단하게"})

AIMessage(content='양자컴퓨팅은 양자역학의 원리를 기반으로 하는 컴퓨팅 방식입니다. 전통적인 컴퓨터는 비트(bit)를 사용하여 정보를 처리하는 반면, 양자컴퓨터는 큐비트(qubit)를 사용합니다. 큐비트는 0과 1의 상태를 동시에 가질 수 있는 중첩(superposition) 상태를 가질 수 있으며, 이는 양자 얽힘(entanglement)과 같은 다른 양자 현상과 결합되어 복잡한 계산을 동시에 수행할 수 있게 합니다.\n\n양자컴퓨터는 특정 문제, 예를 들어 소인수 분해, 최적화 문제, 양자 시뮬레이션 등에서 전통적인 컴퓨터보다 훨씬 빠른 속도로 계산할 수 있는 잠재력을 가지고 있습니다. 그러나 현재 양자컴퓨터는 기술적으로 아직 발전 중이며, 상용화에는 여러 도전 과제가 남아 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 202, 'prompt_tokens': 20, 'total_tokens': 222, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'finish_reason': 'stop', 'logprobs': None}, id='run-d162e760-4a63-4e72-a037-c28010441033-0', usage_metadata={'input_tokens': 20, 'output_tokens': 202, 'total_tokens': 222, 'in

In [15]:
chain2.invoke({"topic": "양자컴퓨팅", "how": "쉽게"})

AIMessage(content="양자컴퓨팅은 양자역학의 원리를 이용하여 정보를 처리하는 새로운 형태의 컴퓨팅입니다. 전통적인 컴퓨터는 비트(bit)를 사용하여 정보를 처리하는데, 비트는 0 또는 1의 두 가지 상태만 가질 수 있습니다. 반면, 양자컴퓨터는 큐비트(qubit)를 사용합니다.\n\n큐비트는 0과 1의 상태를 동시에 가질 수 있는 '중첩(superposition)' 상태를 가질 수 있습니다. 예를 들어, 하나의 큐비트는 0, 1, 또는 그 두 상태의 조합으로 존재할 수 있습니다. 이 덕분에 양자컴퓨터는 여러 계산을 동시에 수행할 수 있는 능력을 가집니다.\n\n또한, 큐비트는 '얽힘(entanglement)'이라는 현상을 통해 서로 연결될 수 있습니다. 얽힌 큐비트는 하나의 큐비트의 상태가 다른 큐비트의 상태에 즉각적으로 영향을 미치게 됩니다. 이 특성은 양자컴퓨터가 복잡한 문제를 더 빠르게 해결할 수 있도록 도와줍니다.\n\n양자컴퓨터는 특정 문제, 예를 들어 소인수 분해나 최적화 문제 등에서 전통적인 컴퓨터보다 훨씬 빠른 속도로 계산할 수 있는 잠재력을 가지고 있습니다. 하지만 현재 양자컴퓨터는 아직 개발 초기 단계에 있으며, 상용화되기까지는 여러 기술적 도전과제가 남아 있습니다. \n\n요약하자면, 양자컴퓨팅은 양자역학의 원리를 활용하여 정보를 처리하는 방식으로, 더 빠르고 효율적인 계산을 가능하게 하는 기술입니다.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 372, 'prompt_tokens': 18, 'total_tokens': 390, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_det

아래는 스트리밍을 출력하는 예시 입니다.

In [19]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능 모델의 학습 원리는 크게 두 가지 단계로 나눌 수 있습니다: **데이터 수집**과 **모델 학습**입니다.

1. **데이터 수집**: 인공지능 모델은 많은 양의 데이터를 필요로 합니다. 이 데이터는 모델이 학습할 수 있는 '경험'을 제공합니다. 예를 들어, 고양이와 개를 구분하는 모델을 만들고 싶다면, 고양이와 개의 사진을 많이 모아야 합니다.

2. **모델 학습**: 데이터를 수집한 후, 모델은 이 데이터를 바탕으로 학습을 시작합니다. 이 과정은 다음과 같은 단계로 이루어집니다.

   - **입력과 출력**: 모델은 입력 데이터(예: 고양이 사진)를 받고, 이에 대한 정답(예: '고양이'라는 레이블)을 학습합니다.
   
   - **예측**: 모델은 처음에는 무작위로 예측을 합니다. 예를 들어, 고양이 사진을 보고 '개'라고 예측할 수 있습니다.
   
   - **오류 계산**: 모델의 예측이 실제 정답과 얼마나 다른지를 계산합니다. 이 차이를 '오류'라고 합니다.
   
   - **학습**: 모델은 이 오류를 바탕으로 자신의 예측을 개선하기 위해 내부 파라미터(가중치)를 조정합니다. 이 과정을 여러 번 반복하면서 점점 더 정확한 예측을 하게 됩니다.

3. **검증**: 모델이 학습한 후에는 새로운 데이터로 테스트하여 얼마나 잘 작동하는지를 확인합니다. 이 단계에서 모델의 성능을 평가하고, 필요하다면 다시 학습을 진행할 수 있습니다.

이러한 과정을 통해 인공지능 모델은 주어진 데이터를 기반으로 패턴을 인식하고, 새로운 데이터에 대해서도 예측을 할 수 있게 됩니다. 쉽게 말해, 많은 경험을 통해 '배우는' 과정이라고 할 수 있습니다.

### 출력파서(Output Parser)


In [20]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [21]:
chain = prompt | model

In [22]:
input = {"topic": "인공지능 모델의 학습 원리"}
answer = chain.invoke(input)

answer 데이터 타입 : AIMessage 타입, 응답결과 데이터가 있는 content, response_metadata 토큰 등 메타 정보 

In [23]:
answer

AIMessage(content='인공지능 모델의 학습 원리를 쉽게 설명하자면, 다음과 같은 단계로 나눌 수 있습니다.\n\n1. **데이터 수집**: 인공지능 모델은 학습하기 위해 많은 데이터를 필요로 합니다. 이 데이터는 이미지, 텍스트, 소리 등 다양한 형태일 수 있습니다.\n\n2. **데이터 전처리**: 수집한 데이터는 종종 정리하고 변환해야 합니다. 예를 들어, 이미지의 크기를 맞추거나, 텍스트를 숫자로 변환하는 등의 작업이 필요합니다.\n\n3. **모델 선택**: 학습할 모델을 선택합니다. 이는 신경망, 결정 트리, 서포트 벡터 머신 등 여러 종류가 있습니다. 각 모델은 특정한 문제에 더 적합할 수 있습니다.\n\n4. **학습 과정**: 모델은 데이터를 통해 패턴을 학습합니다. 이 과정에서 모델은 입력 데이터와 정답(라벨)을 비교하여 오차를 계산합니다. 이 오차를 줄이기 위해 모델의 내부 파라미터(가중치)를 조정합니다. 이 과정을 반복하면서 모델은 점점 더 정확한 예측을 할 수 있게 됩니다.\n\n5. **검증**: 학습이 끝난 후, 모델의 성능을 평가하기 위해 새로운 데이터(검증 데이터)를 사용합니다. 이 데이터는 모델이 학습할 때 사용하지 않은 데이터로, 모델의 일반화 능력을 확인하는 데 중요합니다.\n\n6. **조정 및 개선**: 모델의 성능이 만족스럽지 않다면, 하이퍼파라미터를 조정하거나, 더 많은 데이터를 수집하거나, 다른 모델을 시도하는 등의 방법으로 개선할 수 있습니다.\n\n7. **배포**: 최종적으로 학습된 모델은 실제 환경에 배포되어 사용됩니다. 사용자는 이 모델을 통해 예측이나 분류 등의 작업을 수행할 수 있습니다.\n\n이러한 과정을 통해 인공지능 모델은 데이터를 기반으로 학습하고, 새로운 상황에서도 유용한 예측을 할 수 있게 됩니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 444, 'pro

Chain 에 출력파서를 추가합니다.

In [24]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [25]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
answer2 = chain.invoke(input)

In [26]:
answer2

'인공지능 모델의 학습 원리를 쉽게 설명하자면, 다음과 같은 단계로 이해할 수 있습니다.\n\n1. **데이터 수집**: 인공지능 모델은 학습하기 위해 많은 데이터를 필요로 합니다. 이 데이터는 이미지, 텍스트, 소리 등 다양한 형태일 수 있습니다.\n\n2. **데이터 전처리**: 수집한 데이터는 모델이 이해할 수 있는 형태로 변환해야 합니다. 예를 들어, 이미지의 크기를 조정하거나, 텍스트를 숫자로 변환하는 과정이 필요합니다.\n\n3. **모델 선택**: 학습할 모델을 선택합니다. 이는 신경망, 결정 트리, 서포트 벡터 머신 등 다양한 알고리즘 중 하나일 수 있습니다.\n\n4. **학습**: 모델은 주어진 데이터를 바탕으로 패턴을 학습합니다. 이 과정에서 모델은 입력 데이터와 정답(레이블) 간의 관계를 파악하게 됩니다. 예를 들어, 고양이와 개의 이미지를 구분하는 모델은 각각의 특징을 학습하게 됩니다.\n\n5. **손실 함수**: 모델의 예측이 실제 정답과 얼마나 차이가 있는지를 측정하는 손실 함수를 사용합니다. 이 값을 최소화하는 방향으로 모델의 파라미터(가중치)를 조정합니다.\n\n6. **최적화**: 경량화된 알고리즘(예: 경사 하강법)을 사용하여 손실 함수를 최소화하는 방향으로 모델의 파라미터를 업데이트합니다. 이 과정을 여러 번 반복하여 모델이 점점 더 정확해지도록 합니다.\n\n7. **검증**: 학습이 끝난 후, 모델의 성능을 평가하기 위해 별도의 검증 데이터를 사용합니다. 이를 통해 모델이 새로운 데이터에 대해서도 잘 작동하는지 확인합니다.\n\n8. **배포 및 사용**: 학습이 완료된 모델은 실제 환경에 배포되어 사용됩니다. 사용자가 입력한 데이터를 바탕으로 예측을 하거나 결정을 내리게 됩니다.\n\n이러한 과정을 통해 인공지능 모델은 데이터를 기반으로 학습하고, 새로운 상황에서도 유용한 예측을 할 수 있게 됩니다.'

In [27]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능 모델의 학습 원리를 쉽게 설명하자면, 다음과 같은 단계로 나눌 수 있습니다.

1. **데이터 수집**: 인공지능 모델은 학습하기 위해 많은 데이터를 필요로 합니다. 이 데이터는 이미지, 텍스트, 소리 등 다양한 형태일 수 있습니다.

2. **데이터 전처리**: 수집한 데이터는 종종 정제와 변환이 필요합니다. 예를 들어, 이미지의 크기를 조정하거나, 텍스트를 정리하는 등의 작업을 통해 모델이 이해하기 쉽게 만듭니다.

3. **모델 선택**: 학습할 모델을 선택합니다. 이는 신경망, 결정 트리, 서포트 벡터 머신 등 여러 종류가 있으며, 문제의 특성에 따라 적합한 모델을 선택합니다.

4. **학습 과정**: 모델은 주어진 데이터를 바탕으로 패턴을 학습합니다. 이 과정에서 모델은 입력 데이터와 정답(라벨) 간의 관계를 이해하려고 합니다. 예를 들어, 고양이와 개의 이미지를 구분하는 모델은 각각의 특징을 학습하게 됩니다.

5. **손실 함수**: 모델이 예측한 결과와 실제 정답 간의 차이를 측정하기 위해 손실 함수를 사용합니다. 이 값이 작을수록 모델의 예측이 정확하다는 의미입니다.

6. **최적화**: 손실 값을 최소화하기 위해 모델의 파라미터(가중치)를 조정합니다. 이 과정은 반복적으로 이루어지며, 이를 통해 모델은 점점 더 정확한 예측을 하게 됩니다.

7. **검증**: 학습이 끝난 후, 모델의 성능을 평가하기 위해 별도의 검증 데이터를 사용합니다. 이 단계에서 모델이 실제 상황에서도 잘 작동하는지 확인합니다.

8. **배포 및 활용**: 모델이 충분히 학습하고 검증되면, 실제 환경에 배포하여 사용합니다. 이후에도 새로운 데이터로 모델을 업데이트하거나 개선할 수 있습니다.

이러한 과정을 통해 인공지능 모델은 데이터를 기반으로 학습하고, 주어진 문제를 해결하는 능력을 갖추게 됩니다.

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [28]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

# 변수가 아님, 문자열을 입력할때 {{ }} 사용
{{question}}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4o-mini")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [29]:
print(prompt)

input_variables=['question'] input_types={} partial_variables={} template='\n당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.\n양식은 [FORMAT]을 참고하여 작성해 주세요.\n\n#상황:\n{question}\n\n# 변수가 아님, 문자열을 입력할때 {{ }} 사용\n{{question}}\n\n#FORMAT:\n- 영어 회화:\n- 한글 해석:\n'


In [30]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [31]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화: "Hello! I would like to order, please. Can you recommend any specialties?"
- 한글 해석: "안녕하세요! 주문하고 싶습니다. 추천할 만한 특선 메뉴가 있을까요?"


In [32]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화: "Hello! I would like to see the menu, please."
- 한글 해석: "안녕하세요! 메뉴를 보고 싶어요." 

- 영어 회화: "Can I have the special of the day?"
- 한글 해석: "오늘의 특별 요리를 주문할 수 있을까요?"

- 영어 회화: "Could you recommend a popular dish?"
- 한글 해석: "인기 있는 요리를 추천해 주시겠어요?"

- 영어 회화: "I’ll have the grilled chicken, please."
- 한글 해석: "구운 치킨으로 할게요."

- 영어 회화: "Can I get a side salad with that?"
- 한글 해석: "그것과 함께 사이드 샐러드를 주문할 수 있을까요?"

- 영어 회화: "Could I get a glass of water, please?"
- 한글 해석: "물 한 잔 주실 수 있나요?"

- 영어 회화: "That’s all for now, thank you!"
- 한글 해석: "지금은 그게 다예요, 감사합니다!"

In [33]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:  
Customer: Hi, I’d like to order a pizza. What are your options?  
Server: Sure! We have a variety of toppings. Would you like a small, medium, or large pizza?  
Customer: I’ll have a large pepperoni pizza, please.  
Server: Great choice! Would you like any drinks or sides with that?  
Customer: Yes, I’d like a 2-liter soda and some garlic bread.  
Server: Perfect! Your total comes to $25. Would you like to pay now or upon delivery?  
Customer: I’ll pay now, please.  
Server: Alright, can I get your address?  
Customer: Sure, it’s 123 Main Street.  
Server: Thank you! Your pizza will be delivered in about 30 minutes.

- 한글 해석:  
고객: 안녕하세요, 피자를 주문하고 싶어요. 어떤 옵션이 있나요?  
서버: 물론이죠! 다양한 토핑이 있습니다. 작은 것, 중간 것, 큰 것 중 어떤 것을 원하시나요?  
고객: 큰 페퍼로니 피자로 주세요.  
서버: 좋은 선택이에요! 음료수나 사이드 메뉴를 추가하시겠어요?  
고객: 네, 2리터 탄산음료와 마늘빵 하나 주세요.  
서버: 완벽해요! 총 금액은 25달러입니다. 지금 결제하시겠어요, 아니면 배달 시에 결제하시겠어요?  
고객: 지금 결제할게요.  
서버: 알겠습니다! 주소를 말씀해 주시겠어요?  
고객: 네, 123 메인 스트리트입니다.  
서버: 감사합니다! 피자는 약 30분 후에 배달될 거예요.